In [ ]:
%run graph_helper.ipynb
%run heuristics.ipynb

In [ ]:
import scipy
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from itertools import combinations
import time
from scipy.stats import beta
%matplotlib inline

In [ ]:
obj_functions = [obj_disagreement, obj_polarization, obj_sum]

In [ ]:
DISAGREEMENT = 0
POLARIZATION = 1
BOTH = 2

# Helper functions for measuring heuristic performance

In [ ]:
def compare_algorithms(n, k, G, obj_type, opinions=None):
    if opinions is None:
        s = np.random.rand(n, 1) # if not specified, create random innate opinion vector
    else:
        s = opinions

    L = scipy.sparse.csgraph.laplacian(G, normed=False)
    A = np.linalg.inv(np.identity(n) + L)
    m = num_edges(L, n)
    
    # create copies of innate opinion vector to be used in each heuristic
    # v: k-length array indicating what indices of vertices are changed at kth step
    # op_hat: k-length array indicating if the vertex is changed to 0 or 1
    # max_obj: k-length array indicating the objective function value (polarization or disagreement)
    s_greedy = np.copy(s)
    s_local_innate = np.copy(s)
    s_random = np.copy(s)
    s_partial_random = np.copy(s)
    s_mean = np.copy(s)
    s_deg = np.copy(s)
    s_w_deg = np.copy(s)
    s_double_heuristic = np.copy(s)
    
    v_greedy_arr = np.zeros(k)
    op_hat_greedy_arr = np.zeros(k)
    max_obj_greedy_arr = np.zeros(k)
    
    v_local_innate_arr = np.zeros(k)
    op_hat_local_innate_arr = np.zeros(k)
    max_obj_local_innate_arr = np.zeros(k)
    
    v_random_arr = np.zeros(k)
    op_hat_random_arr = np.zeros(k)
    max_obj_random_arr = np.zeros(k)
    
    v_partial_random_arr = np.zeros(k)
    op_hat_partial_random_arr = np.zeros(k)
    max_obj_partial_random_arr = np.zeros(k)
    
    v_mean_arr = np.zeros(k)
    op_hat_mean_arr = np.zeros(k)
    max_obj_mean_arr = np.zeros(k)
    
    v_deg_arr = np.zeros(k)
    op_hat_deg_arr = np.zeros(k)
    max_obj_deg_arr = np.zeros(k)
    
    v_w_deg_arr = np.zeros(k)
    op_hat_w_deg_arr = np.zeros(k)
    max_obj_w_deg_arr = np.zeros(k)
    
    v_double_heuristic_arr = np.zeros(k)
    op_hat_double_heuristic_arr = np.zeros(k)
    max_obj_double_heuristic_arr = np.zeros(k)

    # populate above arrays by trying various heuristics over G for k vertex budget (adaptively)
    for i in range(k):
        (v_greedy, op_hat_greedy, max_obj_greedy) = choose_greedy_vertex(
            obj_functions[obj_type], A, L, s_greedy, n, m)
        s_greedy[v_greedy] = op_hat_greedy
        v_greedy_arr[i] = v_greedy
        op_hat_greedy_arr[i] = op_hat_greedy
        max_obj_greedy_arr[i] = max_obj_greedy
        
        (v_local_innate, op_hat_local_innate, max_obj_local_innate) = choose_innate_vertex(
            obj_functions[obj_type], A, L, s_local_innate, n, m, obj_type)
        s_local_innate[v_local_innate] = op_hat_local_innate
        v_local_innate_arr[i] = v_local_innate
        op_hat_local_innate_arr[i] = op_hat_local_innate
        max_obj_local_innate_arr[i] = max_obj_local_innate
            
        (v_random, op_hat_random, max_obj_random) = choose_random(
            obj_functions[obj_type], A, L, s_random, n, m)
        s_random[v_random] = op_hat_random
        v_random_arr[i] = v_random
        op_hat_random_arr[i] = op_hat_random
        max_obj_random_arr[i] = max_obj_random
        
        (v_partial_random, op_hat_partial_random, max_obj_partial_random) = choose_partial_random(
            obj_functions[obj_type], A, L, s_partial_random, n, m)
        s_partial_random[v_partial_random] = op_hat_partial_random  
        v_partial_random_arr[i] = v_partial_random
        op_hat_partial_random_arr[i] = op_hat_partial_random
        max_obj_partial_random_arr[i] = max_obj_partial_random
        
        (v_mean, op_hat_mean, max_obj_mean) = choose_mean(obj_functions[obj_type], A, L, s_mean, n, m)
        s_mean[v_mean] = op_hat_mean 
        v_mean_arr[i] = v_mean
        op_hat_mean_arr[i] = op_hat_mean
        max_obj_mean_arr[i] = max_obj_mean
        
        (v_deg, op_hat_deg, max_obj_deg) = choose_max_deg(obj_functions[obj_type], A, L, s_deg, n, m)
        s_deg[v_deg] = op_hat_deg
        v_deg_arr[i] = v_deg
        op_hat_deg_arr[i] = op_hat_deg
        max_obj_deg_arr[i] = max_obj_deg
        
        (v_w_deg, op_hat_w_deg, max_obj_w_deg) = choose_max_weighted_deg(
            obj_functions[obj_type], A, L, s_w_deg, n, m)
        s_w_deg[v_w_deg] = op_hat_w_deg
        v_w_deg_arr[i] = v_w_deg
        op_hat_w_deg_arr[i] = op_hat_w_deg
        max_obj_w_deg_arr[i] = max_obj_w_deg
        
        (v_double_heuristic, op_hat_double_heuristic, max_obj_double_heuristic) = choose_mean_set_opinion(
            obj_functions[obj_type], A, L, s_double_heuristic, n, m, obj_type)
        s_double_heuristic[v_double_heuristic] = op_hat_double_heuristic
        v_double_heuristic_arr[i] = v_double_heuristic
        op_hat_double_heuristic_arr[i] = op_hat_double_heuristic
        max_obj_double_heuristic_arr[i] = max_obj_double_heuristic
    
        original_obj = obj_functions[obj_type](A, L, s, n, m)

    
    return (s, s_greedy, max_obj_greedy_arr, s_local_innate, max_obj_local_innate_arr, s_random, 
            max_obj_random_arr, s_partial_random, max_obj_partial_random_arr, s_mean, max_obj_mean_arr,
            s_deg, max_obj_deg_arr, s_w_deg, max_obj_w_deg_arr, s_double_heuristic, 
            max_obj_double_heuristic_arr, original_obj)

In [ ]:
# Test compare_algorithms on a binomial random graph.
def test_random_graph(n, d, k, obj_type, s = None, G = None):
    if G is None:
        G = make_adj_matrix(n, d)
    
    (s, s_greedy, max_obj_greedy_arr, s_local_innate, max_obj_local_innate_arr, s_random, 
            max_obj_random_arr, s_partial_random, max_obj_partial_random_arr, s_mean, max_obj_mean_arr,
            s_deg, max_obj_deg_arr, s_w_deg, max_obj_w_deg_arr, s_double_heuristic, 
            max_obj_double_heuristic_arr, original_obj) = compare_algorithms(n, k, G, obj_type, opinions = s)
    
    

    print("For k = {}:".format(k))
    print("Innate opinions: \n {} \n".format(s))
    
    print("Greedy algorithm:")
    print("---------------------------------")
    print("New innate opinion vector: \n {}".format(s_greedy))
    print("Objective function: {} \n".format(max_obj_greedy_arr[k-1]))

    print("---------------------------------")
    print("Localized innate:")
    print("---------------------------------")
    print("New innate opinion vector: \n {}".format(s_local_innate))
    print("Objective function: {} \n".format(max_obj_local_innate_arr[k-1]))

    print("---------------------------------")
    print("Randomization:")
    print("---------------------------------")
    print("New innate opinion vector: \n {}".format(s_random))
    print("Objective function: {} \n".format(max_obj_random_arr[k-1]))

    print("---------------------------------")
    print("Mean opinion with partial randomization:")
    print("---------------------------------")
    print("New innate opinion vector: \n {}".format(s_partial_random))
    print("Objective function: {} \n".format(max_obj_partial_random_arr[k-1]))

    print("---------------------------------")
    print("Mean opinion:")
    print("---------------------------------")
    print("New innate opinion vector: \n {}".format(s_mean))
    print("Objective function: {} \n".format(max_obj_mean_arr[k-1]))

    print("---------------------------------")
    print("Max degree:")
    print("---------------------------------")
    print("New innate opinion vector: \n {}".format(s_deg))
    print("Objective function: {} \n".format(max_obj_deg_arr[k-1]))

    print("---------------------------------")
    print("Max weighted degree:")
    print("---------------------------------")
    print("New innate opinion vector: \n {}".format(s_w_deg))
    print("Objective function: {} \n".format(max_obj_w_deg_arr[k-1]))

    print("---------------------------------")
    print("Double heuristic:")
    print("---------------------------------")
    print("New innate opinion vector: \n {}".format(s_double_heuristic))
    print("Objective function: {} \n".format(max_obj_double_heuristic_arr[k-1]))

    print("Original objective: {}".format(original_obj))

In [ ]:
# test_random_graph(10, 0.3, 2, DISAGREEMENT)

In [ ]:
# plots performance of heuristics over k
def plot_obj(k, objs, labels):
    ks = np.arange(1, k + 1)
    plt.figure(figsize = (6, 4))
    lines = []
    for obj in objs:
        line, = plt.plot(ks, obj)
        lines.append(line)
        
    plt.xlabel("k")
    plt.legend(lines, labels)